### 1. Setup & Data Inspection
Imports libraries, connects to DuckDB, and loads all `.parquet` files as views.

In [189]:
import glob
import os

import duckdb as dd

con = dd.connect()

parquet_dir = os.path.join("processed_data", "*.parquet")
parquet_files = glob.glob(parquet_dir)

for file in parquet_files:
    view_name = os.path.splitext(os.path.basename(file))[0]
    con.sql(f"CREATE VIEW '{view_name}' AS SELECT * FROM read_parquet(['{file}'])")

In [190]:
tables_df = con.sql("SHOW TABLES").pl()

print("--- Database Schema ---")

for table_name in tables_df["name"]:
    header = f" TABLE: {table_name.upper()} "
    print("\n" + "=" * len(header))
    print(header)
    print("=" * len(header))

    schema_df = con.sql(f"DESCRIBE '{table_name}'").pl()

    for col_info in schema_df.to_dicts():
        print(f"  - {col_info['column_name']}: {col_info['column_type']}")

--- Database Schema ---

 TABLE: SKETCH_PLANTS 
  - Plant Name: VARCHAR
  - Growth: VARCHAR
  - Soil: VARCHAR
  - Sunlight: VARCHAR
  - Watering: VARCHAR
  - Fertilization Type: VARCHAR

 TABLE: TRUSTWORTHY_PLANTS 
  - scientific_name: VARCHAR
  - common_name: VARCHAR
  - is_toxic: INTEGER
  - light_level: DOUBLE
  - water_need: BIGINT
  - humidity_need: DOUBLE
  - temp_tolerance: DOUBLE
  - has_flowers: VARCHAR
  - description: VARCHAR
  - General care: VARCHAR


### Initial exploration the less trust worthy dataset

In [191]:
con.sql("""
        SELECT *
        FROM SKETCH_PLANTS
        """)

┌─────────────────────┬──────────┬──────────────┬───────────────────┬───────────────────────────────┬────────────────────┐
│     Plant Name      │  Growth  │     Soil     │     Sunlight      │           Watering            │ Fertilization Type │
│       varchar       │ varchar  │   varchar    │      varchar      │            varchar            │      varchar       │
├─────────────────────┼──────────┼──────────────┼───────────────────┼───────────────────────────────┼────────────────────┤
│ Aloe Vera           │ slow     │ sandy        │ indirect sunlight │ Water weekly                  │ Balanced           │
│ Basil               │ fast     │ well-drained │ full sunlight     │ Keep soil evenly moist        │ Organic            │
│ Snake Plant         │ slow     │ well-drained │ indirect sunlight │ Water when soil is dry        │ No                 │
│ Lavender            │ moderate │ sandy        │ full sunlight     │ Let soil dry between watering │ No                 │
│ Cactus        

In [192]:
con.sql("""
        SELECT COUNT(*), growth
        FROM SKETCH_PLANTS
        GROUP BY growth
        """).show()
con.sql("""
        SELECT COUNT(*), soil
        FROM SKETCH_PLANTS
        GROUP BY soil
        """).show()

┌──────────────┬──────────┐
│ count_star() │  Growth  │
│    int64     │ varchar  │
├──────────────┼──────────┤
│          125 │ slow     │
│          170 │ fast     │
│          301 │ moderate │
└──────────────┴──────────┘

┌──────────────┬──────────────┐
│ count_star() │     Soil     │
│    int64     │   varchar    │
├──────────────┼──────────────┤
│           95 │ loamy        │
│           33 │ moist        │
│          274 │ well-drained │
│          169 │ sandy        │
│           25 │ acidic       │
└──────────────┴──────────────┘



In [193]:
con.sql("""
        SELECT COUNT(*), sunlight
        FROM SKETCH_PLANTS
        GROUP BY sunlight
        """).show()

┌──────────────┬───────────────────┐
│ count_star() │     Sunlight      │
│    int64     │      varchar      │
├──────────────┼───────────────────┤
│          416 │ full sunlight     │
│           99 │ partial sunlight  │
│           81 │ indirect sunlight │
└──────────────┴───────────────────┘



In [194]:
con.sql("""
        SELECT COUNT(*), watering
        FROM SKETCH_PLANTS
        GROUP BY watering
        """).show()

┌──────────────┬───────────────────────────────┐
│ count_star() │           Watering            │
│    int64     │            varchar            │
├──────────────┼───────────────────────────────┤
│           21 │ Water when topsoil is dry     │
│          114 │ Water weekly                  │
│           68 │ Water when soil is dry        │
│          106 │ Regular, moist soil           │
│           36 │ Regular watering              │
│           26 │ Let soil dry between watering │
│           19 │ Keep soil consistently moist  │
│           17 │ Water when soil feels dry     │
│           48 │ Keep soil moist               │
│           10 │ Keep soil evenly moist        │
│            7 │ Keep soil slightly moist      │
│           25 │ Regular Watering              │
│           99 │ Regular, well-drained soil    │
├──────────────┴───────────────────────────────┤
│ 13 rows                            2 columns │
└──────────────────────────────────────────────┘



In [195]:
con.sql("""
        SELECT COUNT(*), "Fertilization Type"
        FROM SKETCH_PLANTS
        GROUP BY "Fertilization Type"
        """).show()

┌──────────────┬────────────────────┐
│ count_star() │ Fertilization Type │
│    int64     │      varchar       │
├──────────────┼────────────────────┤
│           83 │ Low-nitrogen       │
│           48 │ No                 │
│          242 │ Organic            │
│            6 │ Acidic             │
│          217 │ Balanced           │
└──────────────┴────────────────────┘



#### Primary Issues Among Data

* The watering column is very messy and some of it is useless.
* Not a lot of information to build a good reccomendation
* Use of common names is unideal

### Explore Fancier Dataset

In [196]:
con.sql("""
        SELECT *
        FROM TRUSTWORTHY_PLANTS
        """)

┌────────────────────────────┬─────────────────────────────────────────────────────────────┬──────────┬─────────────┬────────────┬───────────────┬────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [197]:
con.sql("""
        SELECT COUNT(*), 'Commercial Light levels'
        FROM TRUSTWORTHY_PLANTS
        GROUP BY 'Commercial Light levels'
        """)

┌──────────────┬───────────────────────────┐
│ count_star() │ 'Commercial Light levels' │
│    int64     │          varchar          │
├──────────────┼───────────────────────────┤
│          250 │ Commercial Light levels   │
└──────────────┴───────────────────────────┘

Checking:
is_toxic
light_level
water_need
humidity_need
temp_tolerance

In [198]:
con.sql("""
        SELECT COUNT(*), is_toxic
        FROM TRUSTWORTHY_PLANTS
        GROUP BY is_toxic
        """)

┌──────────────┬──────────┐
│ count_star() │ is_toxic │
│    int64     │  int32   │
├──────────────┼──────────┤
│          245 │        0 │
│            5 │        1 │
└──────────────┴──────────┘

In [199]:
con.sql("""
        SELECT COUNT(*), light_level, scientific_name
        FROM TRUSTWORTHY_PLANTS
        WHERE light_level IS NULL
        GROUP BY light_level, scientific_name
        """)

┌──────────────┬─────────────┬─────────────────┐
│ count_star() │ light_level │ scientific_name │
│    int64     │   double    │     varchar     │
├──────────────┴─────────────┴─────────────────┤
│                    0 rows                    │
└──────────────────────────────────────────────┘

In [200]:
con.sql("""
        SELECT COUNT(*), water_need, scientific_name, common_name
        FROM TRUSTWORTHY_PLANTS
        WHERE water_need is NULL
        GROUP BY water_need, scientific_name,common_name
        """)

┌──────────────┬────────────┬─────────────────┬─────────────┐
│ count_star() │ water_need │ scientific_name │ common_name │
│    int64     │   int64    │     varchar     │   varchar   │
├──────────────┴────────────┴─────────────────┴─────────────┤
│                          0 rows                           │
└───────────────────────────────────────────────────────────┘

In [201]:
con.sql("""
        SELECT COUNT(*), humidity_need, scientific_name
        FROM TRUSTWORTHY_PLANTS
        WHERE humidity_need IS NULL
        GROUP BY humidity_need, scientific_name
        """)

┌──────────────┬───────────────┬─────────────────┐
│ count_star() │ humidity_need │ scientific_name │
│    int64     │    double     │     varchar     │
├──────────────┴───────────────┴─────────────────┤
│                     0 rows                     │
└────────────────────────────────────────────────┘

In [202]:
con.sql("""
        SELECT COUNT(*), temp_tolerance, scientific_name
        FROM TRUSTWORTHY_PLANTS
        WHERE temp_tolerance IS NULL
        GROUP BY temp_tolerance, scientific_name
        """)

┌──────────────┬────────────────┬─────────────────┐
│ count_star() │ temp_tolerance │ scientific_name │
│    int64     │     double     │     varchar     │
├──────────────┴────────────────┴─────────────────┤
│                     0 rows                      │
└─────────────────────────────────────────────────┘

In [203]:
con.sql("""
        SELECT COUNT(*)
        FROM TRUSTWORTHY_PLANTS
        WHERE has_flowers IS NULL
        """)

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│            0 │
└──────────────┘

### General Data Issues
| Issue | Patch Applied |
| :--- | :--- |
| **Source Tags** | Text columns contained citation artifacts like ``. | **Regex Removal:** Removed all instances of `\`. |
| **Unusable Columns** | `nameArabic`, `Family`, `Problems` were irrelevant or too unstructured. | **Drop:** Excluded from the final `.select()` statement. |
| **Range formatting** | Multiple columns used `2_3`, `2,3`, or `2 3` to denote ranges. | **Standardize & Average:** Replaced delimiters with `_`, split strings, and calculated the mean (e.g., `2_3` $\rightarrow$ `2.5`). |


### Column-Specific Issues

#### 1. `light_level` (formerly `brightness`)
* **Issue:** Excel auto-formatting corrupted ranges (e.g., `1-3` became `2022-03-01`).
* **Issue:** Inconsistent descriptive text (e.g., "Bright indirect lights...").
* **Issue:** NULL values for *Philodendron* and *Hydrangea*.
* **Patch:**
    * **Regex:** Reverted dates to ranges (`2022-03-01` $\rightarrow$ `1_3`).
    * **Mapping:** Mapped descriptive text to numeric equivalents (e.g., "Bright indirect" $\rightarrow$ `2`).
    * **Manual Patch:** Assigned `2.5` to *Philodendron* and `1.5` to *Hydrangea* based on botanical needs.

#### 2. `is_toxic` (formerly `Toxicity`)
* **Issue:** 96% of rows were `NULL`.
* **Issue:** Ambiguous text strings (e.g., "No Incriminating Information...").
* **Patch:**
    * **Assumption:** Treated `NULL` as "Safe".
    * **Keyword Logic:** Set to `1` (Toxic) **only** if text contained "toxic" or "poisonous". All else set to `0`.

#### 3. `water_need` (formerly `watering`)
* **Issue:** `NULL` values for 20 rows.
* **Issue:** Invalid `0` values for 2 rows.
* **Patch:**
    * **Botanical Inference:** Used Regex on `scientific_name` to categorize plants into:
        * **Succulents** (*Sansevieria, Haworthia*, etc.) $\rightarrow$ `3` (Low).
        * **Tropicals** (*Ficus, Monstera*, etc.) $\rightarrow$ `2` (Medium).
        * **Thirsty** (*Hydrangea, Ferns*) $\rightarrow$ `1` (High).

#### 4. `humidity_need` (formerly `solHumidity`)
* **Issue:** Text outlier "Average: 25% to 49%".
* **Issue:** `NULL` values for 3 rows.
* **Patch:**
    * **Mapping:** Converted "Average..." text to category `2`.
    * **Manual Patch:** Assigned `1.0` (High) to *Hydrangea/Pachira* and `2.0` to *Philodendron*.

#### 5. `temp_tolerance` (formerly `temperature`)
* **Issue:** Text outlier "Average: 65°F...".
* **Issue:** `NULL` values for 3 rows.
* **Patch:**
    * **Mapping:** Converted "Average..." text to category `2`.
    * **Manual Patch:** Assigned `1.0` (Cool) to *Hydrangea* and `2.0` to *Pachira/Philodendron*.